In [ ]:
## Connect Google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

# data_path = "/content/gdrive/MyDrive/Trope 1 V2"
data_path = "/content/gdrive/MyDrive/Avzdax/Data"

In [1]:
%cd /content/gdrive/MyDrive/Avzdax/Data

[WinError 3] The system cannot find the path specified: '/content/gdrive/MyDrive/Avzdax/Data'
c:\Users\USER\Desktop\box\AVZDAX\Vechicle-Tagging-Analyser


In [ ]:
!pip install face-recognition
!pip install keras-facenet mtcnn
!pip install mediapipe

In [ ]:
!pip install --quiet ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

In [ ]:
import os
os.listdir(f"{data_path}")

In [ ]:
"""
Vehicle-Driver Matching Tester for Colab

Standalone class to test vehicle-driver matching logic without Redis/streaming infrastructure.
Takes batch of entry and exit frames, clusters vehicles, matches drivers, creates alert videos.

Usage in Colab:
    from vehicle_driver_matcher_tester import VehicleDriverMatcherTester

    tester = VehicleDriverMatcherTester(
        entry_frames_path='./entry_frames',
        exit_frames_path='./exit_frames'
    )

    results = tester.run_analysis()
"""

import cv2
import numpy as np
import os
import glob
import torch
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field
from collections import defaultdict
import uuid
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

import mediapipe as mp


@dataclass
class VehicleSnapshot:
    """Single vehicle detection snapshot."""
    vehicle_id: str
    frame_path: str
    frame: np.ndarray
    vehicle_crop: np.ndarray
    driver_crops: List[np.ndarray]
    bbox: Tuple[int, int, int, int]
    timestamp: float
    is_entry: bool
    vehicle_embedding: Optional[np.ndarray] = None
    driver_embedding: Optional[np.ndarray] = None


@dataclass
class VehicleCluster:
    """Clustered vehicle with all its frames."""
    cluster_id: str
    snapshots: List[VehicleSnapshot] = field(default_factory=list)
    vehicle_embedding: Optional[np.ndarray] = None
    driver_embedding: Optional[np.ndarray] = None
    is_entry: bool = True

    def add_snapshot(self, snapshot: VehicleSnapshot):
        self.snapshots.append(snapshot)

    def compute_embeddings(self):
        """Compute average embeddings from all snapshots."""
        if not self.snapshots:
            return

        # Vehicle embeddings
        vehicle_embs = [s.vehicle_embedding for s in self.snapshots if s.vehicle_embedding is not None]
        if vehicle_embs:
            self.vehicle_embedding = np.mean(vehicle_embs, axis=0)
            self.vehicle_embedding /= (np.linalg.norm(self.vehicle_embedding) + 1e-8)

        # Driver embeddings
        driver_embs = [s.driver_embedding for s in self.snapshots if s.driver_embedding is not None]
        if driver_embs:
            self.driver_embedding = np.mean(driver_embs, axis=0)
            self.driver_embedding /= (np.linalg.norm(self.driver_embedding) + 1e-8)


class VehicleDriverMatcherTester:
    """
    Standalone tester for vehicle-driver matching logic.
    No Redis, no streaming - pure batch processing for testing.
    """

    def __init__(
        self,
        entry_frames_path: str,
        exit_frames_path: str,
        output_path: str = './output',
        # Detection parameters
        vehicle_model_path: str = 'yolov8m.pt',
        face_model_path: Optional[str] = f"{data_path}/yolov8s-face.pt",
        reid_model_path: str = f"{data_path}/model/net_19.pth",
        opts_config_path: str = f"{data_path}/model/opts.yaml",
        # Clustering parameters
        vehicle_similarity_threshold: float = 0.7,
        # Matching parameters
        driver_similarity_threshold: float = 0.6,
        overall_match_threshold: float = 0.5,
        # Video parameters
        video_fps: int = 10,
        verbose: bool = True
    ):
        """
        Initialize tester

        Args:
            entry_frames_path: Directory with entry frames (images)
            exit_frames_path: Directory with exit frames (images)
            output_path: Where to save alert videos and logs
            vehicle_model_path: Path to YOLO vehicle detection model
            face_model_path: Path to YOLO face detection model (optional)
            vehicle_similarity_threshold: Threshold for clustering vehicles
            driver_similarity_threshold: Threshold for matching drivers
            overall_match_threshold: Overall threshold for match/mismatch
            video_fps: FPS for alert videos
            verbose: Print detailed logs
        """
        self.entry_frames_path = Path(entry_frames_path)
        self.exit_frames_path = Path(exit_frames_path)
        self.output_path = Path(output_path)
        self.output_path.mkdir(exist_ok=True, parents=True)

        self.vehicle_similarity_threshold = vehicle_similarity_threshold
        self.driver_similarity_threshold = driver_similarity_threshold
        self.overall_match_threshold = overall_match_threshold
        self.video_fps = video_fps
        self.verbose = verbose

        # Load models
        self._log("=" * 80)
        self._log("INITIALIZING VEHICLE-DRIVER MATCHER TESTER")
        self._log("=" * 80)

        from ultralytics import YOLO
        self._log(f"Loading vehicle detector: {vehicle_model_path}")
        self.vehicle_detector = YOLO(vehicle_model_path)

        # Face detection with fallback
        self.face_detector = None


        self.face_recognition = None

        try:
          from keras_facenet import FaceNet
          self.facenet_emb = FaceNet()
        except:
           self._log("FaceNet not installed")

        # try:
        #   from model.load_reid_model import load_model_from_opts
        #   self.vehicle_reid = load_model_from_opts(opts_config_path, ckpt=reid_model_path, remove_classifier=True)
        # except:
        #     self._log("⚠️  Error with loading Vehicle REID Model")
        from model.load_reid_model import load_model_from_opts
        self.vehicle_reid = load_model_from_opts(opts_config_path, ckpt=reid_model_path, remove_classifier=True)

        import os
        try:
            # Force CPU mode to avoid CUDA driver issues
            import os
            # os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

            import face_recognition
            self.face_recognition = face_recognition
            self._log("✅ face_recognition (dlib) loaded as fallback [CPU mode]")
        except ImportError:
            self._log("⚠️  face_recognition not installed")
            self._log("     Install with: pip install face-recognition")
        except RuntimeError as e:
            if "CUDA" in str(e) or "cuda" in str(e).lower():
                self._log("⚠️  CUDA error with face_recognition - will use YOLO only")
                self._log(f"     {str(e)[:80]}...")
            else:
                raise
        except Exception as e:
            self._log(f"⚠️  Could not load face_recognition: {type(e).__name__}")
            self._log(f"     {str(e)[:80]}...")

        # Ultimate fallback: OpenCV Haar Cascade (lightweight, no dependencies)
        self.opencv_face_cascade = None
        if self.face_recognition is None and self.face_detector is None:
            try:
                self.opencv_face_cascade = cv2.CascadeClassifier(
                    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
                )
                self._log("✅ OpenCV Haar Cascade loaded as fallback (basic detection)")
            except Exception as e:
                self._log(f"⚠️  No face detection available: {e}")
                self._log("     Face detection will be limited")

        # Statistics
        self.stats = {
            'entry_frames_processed': 0,
            'exit_frames_processed': 0,
            'entry_vehicles_detected': 0,
            'exit_vehicles_detected': 0,
            'entry_clusters': 0,
            'exit_clusters': 0,
            'matches_found': 0,
            'mismatches_detected': 0,
            'no_match_found': 0
        }

        self._log("✅ Initialization complete\n")

    def _log(self, message: str):
        """Print log message if verbose."""
        if self.verbose:
            print(message)

    # ==================== MAIN PIPELINE ====================

    def run_analysis(self) -> Dict[str, Any]:
        """
        Run complete analysis pipeline.

        Returns:
            Dictionary with results and statistics
        """
        self._log("\n" + "=" * 80)
        self._log("STARTING ANALYSIS PIPELINE")
        self._log("=" * 80 + "\n")

        # Step 1: Load and process entry frames
        self._log("📥 STEP 1: Processing entry frames...")
        entry_snapshots = self._process_frames_batch(self.entry_frames_path, is_entry=True)
        self.stats['entry_frames_processed'] = len(list(self.entry_frames_path.glob('*.jpg'))) + len(list(self.entry_frames_path.glob('*.png')))
        self.stats['entry_vehicles_detected'] = len(entry_snapshots)
        self._log(f"   Found {len(entry_snapshots)} vehicle snapshots in entry frames\n")

        # Step 2: Load and process exit frames
        self._log("📤 STEP 2: Processing exit frames...")
        exit_snapshots = self._process_frames_batch(self.exit_frames_path, is_entry=False)
        self.stats['exit_frames_processed'] = len(list(self.exit_frames_path.glob('*.jpg'))) + len(list(self.exit_frames_path.glob('*.png')))
        self.stats['exit_vehicles_detected'] = len(exit_snapshots)
        self._log(f"   Found {len(exit_snapshots)} vehicle snapshots in exit frames\n")

        # Step 3: Cluster entry vehicles
        self._log("🔗 STEP 3: Clustering entry vehicles...")
        entry_clusters = self.cluster_vehicles_large_batch(entry_snapshots)
        self.stats['entry_clusters'] = len(entry_clusters)
        self._log(f"   Created {len(entry_clusters)} entry vehicle clusters\n")

        # Step 4: Cluster exit vehicles
        self._log("🔗 STEP 4: Clustering exit vehicles...")
        exit_clusters = self.cluster_vehicles_large_batch(exit_snapshots)
        self.stats['exit_clusters'] = len(exit_clusters)
        self._log(f"   Created {len(exit_clusters)} exit vehicle clusters\n")

        # Step 5: Match vehicles and verify drivers
        self._log("🔍 STEP 5: Matching vehicles and verifying drivers...")
        match_results = self._match_and_verify(entry_clusters, exit_clusters)
        self._log(f"   Processed {len(match_results)} exit vehicles\n")

        # Step 6: Generate summary
        self._log("=" * 80)
        self._log("ANALYSIS COMPLETE")
        self._log("=" * 80)
        self._print_summary()

        return {
            'entry_clusters': entry_clusters,
            'exit_clusters': exit_clusters,
            'match_results': match_results,
            'stats': self.stats
        }

    # ==================== FRAME PROCESSING ====================

    def _process_frames_batch(self, frames_path: Path, is_entry: bool) -> List[VehicleSnapshot]:
        """
        Process all frames in a directory.

        Returns:
            List of VehicleSnapshot objects
        """
        snapshots = []

        # Get all image files
        image_files = sorted(
            list(frames_path.glob('*.jpg')) +
            list(frames_path.glob('*.png')) +
            list(frames_path.glob('*.jpeg'))
        )

        if not image_files:
            self._log(f"   ⚠️  No images found in {frames_path}")
            return snapshots

        self._log(f"   Processing {len(image_files)} images...")

        for img_path in image_files:
            try:
                # Load frame
                frame = cv2.imread(str(img_path))
                if frame is None:
                    self._log(f"   ⚠️  Failed to load: {img_path.name}")
                    continue

                # Detect vehicles
                vehicles = self._detect_vehicles(frame)

                if not vehicles:
                    self._log(f"   ℹ️  No vehicles in: {img_path.name}")
                    continue

                # Process each vehicle
                for vehicle in vehicles:
                    bbox = vehicle['bbox']
                    x1, y1, x2, y2 = bbox

                    # Crop vehicle
                    vehicle_crop = frame[y1:y2, x1:x2]
                    if vehicle_crop.size == 0:
                        continue

                    # Detect driver faces
                    driver_crops = self._detect_driver_in_vehicle(frame, bbox)

                    if not driver_crops:
                        self._log(f"   ℹ️  No driver face in vehicle from {img_path.name}")
                        continue

                    # Extract embeddings
                    vehicle_emb = self._extract_vehicle_embedding(vehicle_crop)
                    driver_emb = self._extract_driver_embedding(driver_crops)   #potential error introduced here if you find more than one person in the vehicle.
                                                                                # The search region is looking at both left and right side of the car.
                                                                                # At entry point and exit point, we should only search the right side of the car as the perfective doesn't change.

                    # Create snapshot
                    snapshot = VehicleSnapshot(
                        vehicle_id=str(uuid.uuid4())[:8],
                        frame_path=str(img_path),
                        frame=frame.copy(),
                        vehicle_crop=vehicle_crop,
                        driver_crops=driver_crops,
                        bbox=bbox,
                        timestamp=os.path.getmtime(img_path),
                        is_entry=is_entry,
                        vehicle_embedding=vehicle_emb,
                        driver_embedding=driver_emb
                    )

                    snapshots.append(snapshot)
                    self._log(f"   ✅ Vehicle snapshot from {img_path.name}: {len(driver_crops)} faces detected")

            except Exception as e:
                self._log(f"   ❌ Error processing {img_path.name}: {e}")
                import traceback
                traceback.print_exc()
                continue

        return snapshots

    def _detect_vehicles(self, frame: np.ndarray) -> List[Dict]:
        """Detect vehicles in frame."""
        try:
            results = self.vehicle_detector(
                frame,
                verbose=False,
                classes=[2, 3, 5, 7],  # car, motorcycle, bus, truck
                conf=0.5
            )

            vehicles = []
            for result in results:
                if not hasattr(result, 'boxes') or result.boxes is None:
                    continue

                for box in result.boxes:
                    try:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                        if x2 <= x1 or y2 <= y1:
                            continue

                        vehicles.append({
                            'bbox': (x1, y1, x2, y2),
                            'confidence': float(box.conf[0])
                        })
                    except:
                        continue

            return vehicles

        except Exception as e:
            self._log(f"   ❌ Vehicle detection error: {e}")
            return []

    def _detect_driver_in_vehicle (
        self,
        frame: np.ndarray,
        vehicle_bbox: Tuple[int, int, int, int]
    ) -> List[np.ndarray]:
        """
        Detect driver faces in vehicle using multi-region strategy.
        """
        x1, y1, x2, y2 = vehicle_bbox
        vehicle_width = x2 - x1
        vehicle_height = y2 - y1

        # Try multiple regions
        search_regions = [
            # Left upper (driver side left-hand traffic)
            frame[y1:int(y1 + vehicle_height * 0.6), x1:int(x1 + vehicle_width * 0.6)],
            # Right upper (driver side right-hand traffic)
            frame[y1:int(y1 + vehicle_height * 0.6), int(x1 + vehicle_width * 0.4):x2],
            # Center upper
            frame[y1:int(y1 + vehicle_height * 0.5), int(x1 + vehicle_width * 0.25):int(x2 - vehicle_width * 0.25)],
            Full vehicle
            frame[y1:y2, x1:x2]
        ]

        for region in search_regions:
            if region.size == 0:
                continue

            faces = self._detect_faces_in_region(region)
            if len(faces) > 0:
                return faces

        return []

    def _detect_faces_in_region(self, region: np.ndarray) -> List[np.ndarray]:
        """Detect faces in a region with YOLO + dlib + OpenCV fallback."""
        # Try YOLO first
        if self.face_detector is not None:
            try:
                results = self.face_detector(region, verbose=False, conf=0.3)

                face_crops = []
                for result in results:
                    if not hasattr(result, 'boxes') or result.boxes is None:
                        continue

                    for box in result.boxes:
                        try:
                            coords = box.xyxy[0].cpu().numpy() if hasattr(box.xyxy[0], 'cpu') else box.xyxy[0]
                            x1, y1, x2, y2 = map(int, coords)

                            if x2 <= x1 or y2 <= y1:
                                continue

                            h, w = region.shape[:2]
                            x1, y1 = max(0, x1), max(0, y1)
                            x2, y2 = min(w, x2), min(h, y2)

                            face_crop = region[y1:y2, x1:x2]
                            if face_crop.size > 0 and self._are_eyes_and_nose_visible_from_crop(face_crop):
                                face_crops.append(face_crop)
                        except:
                            continue

                if len(face_crops) > 0:
                    return face_crops
            except:
                pass

        # Fallback to dlib
        if self.face_recognition is not None:
            try:
                rgb_region = cv2.cvtColor(region, cv2.COLOR_BGR2RGB)
                face_locations = self.face_recognition.face_locations(
                    rgb_region,
                    model="hog",
                    number_of_times_to_upsample=0
                )

                face_crops = []
                for (top, right, bottom, left) in face_locations:
                    face_crop = region[top:bottom, left:right]
                    if face_crop.size > 0 and self._check_face_quality(face_crop):
                        face_crops.append(face_crop)

                if len(face_crops) > 0:
                    return face_crops
            except:
                pass

        # Ultimate fallback: OpenCV Haar Cascade
        if self.opencv_face_cascade is not None:
            try:
                gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
                faces = self.opencv_face_cascade.detectMultiScale(
                    gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30)
                )

                face_crops = []
                for (x, y, w, h) in faces:
                    face_crop = region[y:y+h, x:x+w]
                    if face_crop.size > 0 and self._check_face_quality(face_crop):
                        face_crops.append(face_crop)

                return face_crops
            except:
                pass

        return []

    def _check_face_quality(self, face_crop: np.ndarray) -> bool:
        """Basic face quality check."""
        try:
            h, w = face_crop.shape[:2]
            if h < 20 or w < 20:
                return False

            gray = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY) if len(face_crop.shape) == 3 else face_crop

            # Check std deviation (reject uniform regions)
            if np.std(gray) < 10:
                return False

            # Check brightness
            brightness = np.mean(gray)
            if brightness < 15 or brightness > 240:
                return False

            return True
        except:
            return False

    def _are_eyes_and_nose_visible_from_crop(self, face_crop, visibility_threshold=0.5, static_image_mode=True ):

      """
      Detect face landmarks using MediaPipe and check if both eyes and nose are visible.
      """
      try:
        mp_face_mesh = mp.solutions.face_mesh
      except:
        print("Error loading mediapipe")

      if face_crop is None or face_crop.size == 0:
          return False

      h, w, _ = face_crop.shape

      face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

      with mp_face_mesh.FaceMesh(
          static_image_mode=static_image_mode,
          max_num_faces=1,
          refine_landmarks=True,
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5
      ) as face_mesh:

          results = face_mesh.process(face_rgb)

          if not results.multi_face_landmarks:
              return False

          face_landmarks = results.multi_face_landmarks[0]

          # Landmark indices
          LEFT_EYE = 33
          RIGHT_EYE = 263
          NOSE = 1

          for idx in [LEFT_EYE, RIGHT_EYE, NOSE]:
              lm = face_landmarks.landmark[idx]

              # Normalized coordinate check
              if not (0.0 <= lm.x <= 1.0 and 0.0 <= lm.y <= 1.0):
                  return False

              # Pixel coordinate check
              x_px = int(lm.x * w)
              y_px = int(lm.y * h)

              if x_px < 0 or x_px >= w or y_px < 0 or y_px >= h:
                  return False

              #   visibility / presence checks
              if hasattr(lm, "visibility") and lm.visibility < visibility_threshold:
                  return False

              if hasattr(lm, "presence") and lm.presence < visibility_threshold:
                  return False

          return True

    # ==================== EMBEDDING EXTRACTION ====================
    def _extract_driver_embedding(self, driver_crops: List[np.ndarray])-> np.ndarray:
      try:
        if not driver_crops:
          return np.zeros(512, dtype=np.float64)
        embeddings_list = []

        for face in driver_crops:
          print(f"face shape: {face.shape}")
          face = cv2.resize(face, (128, 128))
          face = face.astype('float64')
          face = np.expand_dims(face, axis=0)
          embedding = self.facenet_emb.embeddings(face)
          embeddings_list.append(embedding[0])

        # Average all embeddings
        avg_embedding = np.mean(embeddings_list, axis=0)
        avg_embedding = avg_embedding / (np.linalg.norm(avg_embedding))

        return avg_embedding.astype(np.float64)

      except Exception as e:
            self._log(f"   ❌ Driver embedding error: {e}")
            return np.zeros(512, dtype=np.float64)


    def _extract_vehicle_embedding(self, vehicle_crop: np.ndarray) -> np.ndarray:
        """Extract vehicle ReID embedding using color histogram."""
        device = "cuda"
        try:
            # Resize
            resized = cv2.resize(cv2.cvtColor(vehicle_crop, cv2.COLOR_BGR2RGB), (224, 224))
            resized = resized.transpose(2, 0, 1)
            resized = torch.from_numpy(resized).float().unsqueeze(0).to(device)

            try:
                model = self.vehicle_reid.eval().to(device)
                vehicle_embedding = model(resized)
                vehicle_embedding = vehicle_embedding.detach().cpu().numpy()
                vehicle_embedding = vehicle_embedding / (np.linalg.norm(vehicle_embedding))

            except ImportError:
               vehicle_embedding = np.array([])

            return vehicle_embedding.astype(np.float64)

        except Exception as e:
            self._log(f"   ❌ Vehicle embedding error: {e}")
            return np.zeros(512, dtype=np.float64)



    # ==================== CLUSTERING ====================

    def cluster_vehicles_large_batch(self, snapshots: List[VehicleSnapshot]) -> List[VehicleCluster]:

      if not snapshots:
          return []

      clusters: List[VehicleCluster] = []
      centroids: List[np.ndarray] = []  # each is (D,)

      for snapshot in snapshots:
          emb = snapshot.vehicle_embedding  # already normalized

          # First cluster
          if not centroids:
              cluster = VehicleCluster(
                  cluster_id=f"{snapshot.vehicle_id}",
                  is_entry=snapshot.is_entry
              )
              cluster.add_snapshot(snapshot)
              cluster.compute_embeddings()

              clusters.append(cluster)
              centroids.append(cluster.vehicle_embedding)  # normalized
              continue

          # Stack centroids for vectorized similarity
          centroid_matrix = np.vstack(centroids)  # (K, D)
          similarities = self.cosine_similarity_to_centroids(
              emb,
              centroid_matrix
          )

          best_idx = int(np.argmax(similarities))
          best_score = similarities[best_idx]

          if best_score >= self.vehicle_similarity_threshold:
              # Assign to existing cluster
              cluster = clusters[best_idx]
              cluster.add_snapshot(snapshot)

              # Incremental centroid update (still normalized later)
              n = len(cluster.snapshots)
              updated_centroid = (
                  (centroids[best_idx] * (n - 1) + emb) / n
              )

              #re-normalize centroid
              centroids[best_idx] = updated_centroid / np.linalg.norm(updated_centroid)

          else:
              # Create new cluster
              cluster = VehicleCluster(
                  cluster_id=f"{snapshot.vehicle_id}",
                  is_entry=snapshot.is_entry
              )
              cluster.add_snapshot(snapshot)
              cluster.compute_embeddings()

              clusters.append(cluster)
              centroids.append(cluster.vehicle_embedding)  # already normalized

              self._log(f"   Cluster {cluster.cluster_id}: {len(cluster.snapshots)} snapshots")

      return clusters


    # ==================== MATCHING & VERIFICATION ====================

    def _match_and_verify(
        self,
        entry_clusters: List[VehicleCluster],
        exit_clusters: List[VehicleCluster]
    ) -> List[Dict[str, Any]]:
        """
        Match exit vehicles to entry vehicles and verify drivers.
        """
        results = []

        for exit_cluster in exit_clusters:
            self._log(f"\n🚗 Processing exit vehicle: {exit_cluster.cluster_id}")

            # Find matching entry vehicle
            best_match = None
            best_driver_score = 0.0

            for entry_cluster in entry_clusters:

                driver_score = self.compute_cosine_similarity(
                    exit_cluster.driver_embedding,
                    entry_cluster.driver_embedding
                )
                print(f"driver_score: {driver_score}")

                if driver_score > best_driver_score:
                    best_driver_score = driver_score
                    best_match = entry_cluster

            if best_match is None or best_driver_score < self.driver_similarity_threshold:
                self._log(f"   ⚠️  No matching entry driver found (best score: {best_driver_score:.3f})")
                self.stats['no_match_found'] += 1

                results.append({
                    'exit_cluster': exit_cluster,
                    'entry_cluster': None,
                    'vehicle_score': best_driver_score,
                    'driver_score': 0.0,
                    'overall_score': 0.0,
                    'is_match': False,
                    'reason': 'no_entry_driver_found'
                })
                continue

            self._log(f"   ✅ Matched to entry driver: {best_match.cluster_id} (score: {best_driver_score:.3f})")

            # Verify vehicle
            vehicle_score = self.compute_cosine_similarity(
                exit_cluster.vehicle_embedding,
                best_match.vehicle_embedding
            )

            # Calculate overall score
            overall_score = (0.4 * best_driver_score) + (0.6 * vehicle_score)

            is_match = (
                driver_score >= self.driver_similarity_threshold and
                overall_score >= self.overall_match_threshold
            )

            self._log(f"   vehicle similarity: {vehicle_score:.3f}")
            self._log(f"   Overall score: {overall_score:.3f}")

            if is_match:
                self._log(f"   ✅ MATCH: Same driver")
                self.stats['matches_found'] += 1
            else:
                self._log(f"   ❌ MISMATCH: Different driver!")
                self.stats['mismatches_detected'] += 1

                # Create alert video
                self._create_mismatch_alert_video(
                    entry_cluster=best_match,
                    exit_cluster=exit_cluster,
                    vehicle_score=vehicle_score,
                    driver_score=best_driver_score,
                    overall_score=overall_score
                )

            results.append({
                'exit_cluster': exit_cluster,
                'entry_cluster': best_match,
                'driver_score': best_driver_score,
                'vehicle_score': vehicle_score,
                'overall_score': overall_score,
                'is_match': is_match,
                'reason': 'match' if is_match else 'driver_mismatch'
            })

        return results

    import numpy as np

    def cosine_similarity_to_centroids(self, 
        embedding: np.ndarray,
        centroids: np.ndarray
    ) -> np.ndarray:
        """
        embedding: (D,)
        centroids: (K, D)
        returns: (K,)
        """
        if embedding.ndim == 2:
          embedding = embedding.squeeze(0)  # (D,)
        return np.dot(centroids, embedding)


    def compute_cosine_similarity(self, vec1, vec2):
      vec1 = vec1.reshape(1,-1)
      vec2 = vec2.reshape(1,-1)

      similarity = cosine_similarity(vec1, vec2)
      return float(similarity[0][0])

  

    def _print_summary(self):
        """Print analysis summary."""
        self._log("\n📊 ANALYSIS SUMMARY:")
        self._log(f"   Entry frames processed: {self.stats['entry_frames_processed']}")
        self._log(f"   Exit frames processed: {self.stats['exit_frames_processed']}")
        self._log(f"   Entry vehicles detected: {self.stats['entry_vehicles_detected']}")
        self._log(f"   Exit vehicles detected: {self.stats['exit_vehicles_detected']}")
        self._log(f"   Entry clusters: {self.stats['entry_clusters']}")
        self._log(f"   Exit clusters: {self.stats['exit_clusters']}")
        self._log(f"   ✅ Matches found: {self.stats['matches_found']}")
        self._log(f"   ❌ Mismatches detected: {self.stats['mismatches_detected']}")
        self._log(f"   ⚠️  No match found: {self.stats['no_match_found']}")
        self._log(f"\n📁 Output directory: {self.output_path}")


In [ ]:
tester = VehicleDriverMatcherTester(
            entry_frames_path=f"{data_path}/entry_frames",
            exit_frames_path=f"{data_path}/exit_frames",
            vehicle_similarity_threshold=0.7,
            driver_similarity_threshold=0.6,
            overall_match_threshold=0.5
        )

results = tester.run_analysis()

INITIALIZING VEHICLE-DRIVER MATCHER TESTER
Loading vehicle detector: yolov8m.pt


/usr/local/lib/python3.12/dist-packages/torch/hub.py:335: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/XingangPan/IBN-Net/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet50_ibn_a-d9d0bb7b.pth" to /root/.cache/torch/hub/checkpoints/resnet50_ibn_a-d9d0bb7b.pth


100%|██████████| 97.7M/97.7M [00:01<00:00, 56.3MB/s]


✅ face_recognition (dlib) loaded as fallback [CPU mode]
✅ Initialization complete


STARTING ANALYSIS PIPELINE

📥 STEP 1: Processing entry frames...
   Processing 85 images...
   ℹ️  No driver face in vehicle from frame_000011.jpg
   ℹ️  No driver face in vehicle from frame_000012.jpg
   ℹ️  No driver face in vehicle from frame_000013.jpg
   ℹ️  No driver face in vehicle from frame_000014.jpg
   ℹ️  No driver face in vehicle from frame_000015.jpg
   ℹ️  No driver face in vehicle from frame_000016.jpg
   ℹ️  No driver face in vehicle from frame_000017.jpg
   ℹ️  No driver face in vehicle from frame_000018.jpg
   ℹ️  No driver face in vehicle from frame_000019.jpg
   ℹ️  No driver face in vehicle from frame_000020.jpg
   ℹ️  No driver face in vehicle from frame_000021.jpg
   ℹ️  No driver face in vehicle from frame_000022.jpg
   ℹ️  No driver face in vehicle from frame_000023.jpg
   ℹ️  No driver face in vehicle from frame_000024.jpg
   ℹ️  No driver face in vehicle from frame_000025.jpg